### Questions

1. What is the total appreciation over the period and what is the average annual return?
2. Was it better to invest or to have cash in that period? And if to invest, then in one lump sum or by the DCA method?

### Import libraries

I'll import the libraries first. `Yfinance` to get the data, `pandas` and `datetime` to work with datasets and `plotly` for visualization.

In [1]:
# Import libraries

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

### Preparing data

The `download_and_process_data` function gets the data from yahoo finance and extracts only the close values for each day.

In [2]:
# Getting data from finance.yahoo.com

def download_and_process_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    tickers.sort()
    data_adj = data['Close'].copy()
    data_adj['Date'] = data_adj.index
    data_adj = data_adj.reset_index(drop=True)
    print("\n")
    return data_adj

The `validate_tickers` function checks that tickers are entered in the correct format.

In [3]:
# Ticker validation

def validate_tickers(tickers):
    tickers = [ticker.strip() for ticker in tickers]
    if not all(tickers):
        raise ValueError("Tickers cannot be empty or contain only spaces.")
    return tickers

## Calculations

The `calculate_dca_metrics` function calculates several metrics related to Dollar-Cost Averaging (DCA) for a list of stock tickers over a given date range. It assumes you have adjusted historical price data for the tickers and that DCA is done daily with a fixed dollar amount (e.g., $1 per day).

In [4]:
# Preparing variables for DCA (dollar cost averaging, buying for same amount of dollars every day when asset is tradeable)

def calculate_dca_metrics(data_adj, tickers, start_date, end_date):
    newest_prices = [data_adj[ticker].dropna().iloc[-1] for ticker in tickers]
    amount_bought = [sum(1/data_adj[ticker].dropna()) for ticker in tickers]
    value_in_dollars = [amount_bought[i] * newest_prices[i] for i in range(len(tickers))]
    count_dca = [len(data_adj[ticker].dropna()) for ticker in tickers]
    profit = [(value_in_dollars[i] / count_dca[i] - 1) * 100 for i in range(len(tickers))]
    change_dca = [(value_in_dollars[i] / count_dca[i]) for i in range(len(tickers))]
    diff = end_date - start_date
    diff = diff.days
    cagr = [((value_in_dollars[i] / count_dca[i]) ** (1 / (diff/365)) - 1) * 100 for i in range(len(tickers))]

    return newest_prices, amount_bought, value_in_dollars, count_dca, profit, change_dca, cagr

The `print_dca_results` function summarizes and prints key Dollar-Cost Averaging (DCA) metrics for each ticker, including units bought, total value, profit, investment efficiency, and CAGR.

In [5]:
# Print DCA results

def print_dca_results(tickers, amount_bought, value_in_dollars, count_dca, profit, newest_prices, change_dca, cagr):
    for i in range(len(tickers)):
        print(f"Each dollar invested in {tickers[i]} daily during whole period makes {amount_bought[i]:.4f} units ({value_in_dollars[i]:.2f}$), total amount invested {count_dca[i]}$, profit is {profit[i]:.2f}%, {change_dca[i]:.2f} times invested value, with CAGR {cagr[i]:.2f}%.")
        print(f"So you needed to invest {1/amount_bought[i]:.2f}$ per day to have 1 unit today, worth {newest_prices[i]:.2f}$.\n")

The `calculate_and_print_equalization_metrics` function normalizes each ticker's price data to start at 1 (based on its first valid price) and calculates how many times each $1 invested on the first trading day has grown by the end of the period. It then prints and returns the normalized data, the final relative values, percentage profits, and a list of final multipliers, highlighting the best-performing ticker.

In [ ]:
# Preparing variables for equalization (price in first day of trading is set to value 1) and print results

def calculate_and_print_equalization_metrics(data_adj, tickers):
    first_prices = {}
    x = -1
    y = []
    price_2 = []
    data_adj_2 = data_adj.copy()
    for column in data_adj_2.columns[:-1]:
        first_valid_value = data_adj_2.loc[data_adj_2[column].notna(), column].iloc[0]
        first_prices[column] = first_valid_value
    for column, first_price in first_prices.items():
        data_adj_2[column] = data_adj_2[column] / first_price
    newest_prices_2 = [data_adj_2[i].dropna().iloc[-1] for i in tickers]
    print("Today value of first trading day in our date frame is:")
    for i, price in zip(tickers, newest_prices_2):
        print(f"{price:.2f} times of invested value for {i}")
        if price > x:
            x = price
            y = i
        else:
            pass
        price_2.append(price)
    print(f"The best investment in the period under review was {y}, where each $1 invested is worth ${x:.2f} today.")
    print("\nProfit is:")
    profit_percentages = [(data_adj_2[i].dropna().iloc[-1]-1)*100 for i in tickers]
    for i, profit in zip(tickers,profit_percentages):
        print(f"{profit:.2f}% for {i}")
    return data_adj_2, newest_prices_2, profit_percentages, price_2


## Visualization

The `create_equalization_plot` function creates a chart that shows the price of individual tickers over time.

In [ ]:
# Creating price chart

def create_equalization_plot(data_adj, tickers, start_date, end_date):
    fig_close_core = px.line(data_adj,
                            x='Date',
                            y=tickers,
                            title='Prices from %s to %s' %(start_date, end_date),
                            width=800,
                            height=600)
    fig_close_core.show()

The `create_adjusted_plot` function creates and displays two interactive line plots showing the normalized price evolution (starting from 1) of selected tickers over time, starting from a given date.

In [8]:
# Creating charts with equalization

def create_adjusted_plot(data_adj_2, tickers, start_date):
    fig_close = go.Figure()
    for ticker in tickers:
        fig_close.add_trace(go.Scatter(x=data_adj_2['Date'], y=data_adj_2[ticker], mode='lines', name=ticker))
    fig_close.update_layout(title='Adjusted %s value = 1' %start_date, width=800, height=600, hovermode="x unified")
    fig_close.show()
    print("\n")
    fig_close = px.line(data_adj_2, x='Date', y=tickers, title='Adjusted %s value = 1' %start_date, hover_data={'Date': '|%B %d, %Y'}, width=800, height=600)
    fig_close.show()
    print("\n")

## What was better to do?

The `compare_strategies` function compares Dollar-Cost Averaging (DCA) and lump-sum investing strategies for each ticker over a given period based on CAGR and normalized price growth. It shows which strategy would have been more effective—or whether avoiding investment altogether was better—depending on the performance.

In [9]:
# Comparing with strategy is better

def compare_strategies(tickers, change_dca, newest_prices_2, start_date, end_date, cagr, price_2):
    print(f"In chosen period {start_date} to {end_date} it was better to:")
    for i in range(len(tickers)):

        if (cagr[i]) < 0 and (price_2[i] < 1):
            print(f"Don't buy {tickers[i]} and stay in dollar.") 
        elif change_dca[i] > newest_prices_2[i]:
            print(f"Buy {tickers[i]} with DCA strategy.")    
        else:
            print(f"Make a one-time purchase of {tickers[i]}.")

## Running wholescript

This function serves as the main script that runs the full analysis pipeline for comparing investment strategies over a user-specified period. It takes user input for tickers and dates, downloads and processes historical price data, calculates DCA metrics, visualizes performance, and prints a strategic comparison between DCA and lump-sum investing. Ultimately, it helps the user determine which strategy would have yielded better results for each asset.

In [11]:
# Run the whole code
# Example usage
# tickers = [ETH-USD,MSTR,TSLA,NVDA,PLTR,BTC-USD,^GSPC,^IXIC,GC=F]
# start_date = "2024-01-01"
# end_date = "2025-04-23"

if __name__ == "__main__":
  # user entries
  tickers = input("Enter values separated by a comma: ").split(",")
  tickers = validate_tickers(tickers)

  date_format = "%Y-%m-%d"
  start_date = datetime.strptime(input("Enter the start date (YYYY-MM-DD): "), date_format).date()
  end_date = datetime.strptime(input("Enter the end date (YYYY-MM-DD): "), date_format).date()

  # processing data
  data_adj = download_and_process_data(tickers, start_date, end_date)

  # calculations for DCA
  newest_prices, amount_bought, value_in_dollars, count_dca, profit, change_dca, cagr = calculate_dca_metrics(data_adj, tickers, start_date, end_date)
  print_dca_results(tickers, amount_bought, value_in_dollars, count_dca, profit, newest_prices, change_dca, cagr)

  # graphs and visualizations
  create_equalization_plot(data_adj, tickers, start_date, end_date)
  data_adj_2, newest_prices_2, profit_percentages, price_2 = calculate_and_print_equalization_metrics(data_adj, tickers)
  create_adjusted_plot(data_adj_2, tickers, start_date)

  # comparison of strategies
  compare_strategies(tickers, change_dca, newest_prices_2, start_date, end_date, cagr, price_2)

[*********************100%***********************]  9 of 9 completed




Each dollar invested in BTC-USD daily during whole period makes 0.0070 units (658.52$), total amount invested 478$, profit is 37.77%, 1.38 times invested value, with CAGR 27.72%.
So you needed to invest 141.90$ per day to have 1 unit today, worth 93441.89$.

Each dollar invested in ETH-USD daily during whole period makes 0.1724 units (303.05$), total amount invested 478$, profit is -36.60%, 0.63 times invested value, with CAGR -29.39%.
So you needed to invest 5.80$ per day to have 1 unit today, worth 1757.33$.

Each dollar invested in GC=F daily during whole period makes 0.1324 units (450.42$), total amount invested 328$, profit is 37.32%, 1.37 times invested value, with CAGR 27.40%.
So you needed to invest 7.55$ per day to have 1 unit today, worth 3400.80$.

Each dollar invested in MSTR daily during whole period makes 2.2116 units (758.65$), total amount invested 327$, profit is 132.00%, 2.32 times invested value, with CAGR 90.15%.
So you needed to invest 0.45$ per day to have 1 uni

Today value of first trading day in our date frame is:
2.12 times of invested value for BTC-USD
0.75 times of invested value for ETH-USD
1.65 times of invested value for GC=F
5.01 times of invested value for MSTR
2.05 times of invested value for NVDA
5.67 times of invested value for PLTR
0.96 times of invested value for TSLA
1.11 times of invested value for ^GSPC
1.10 times of invested value for ^IXIC
The best investment in the period under review was PLTR, where each $1 invested is worth $5.67 today.

Profit is:
111.56% for BTC-USD
-25.29% for ETH-USD
64.74% for GC=F
400.66% for MSTR
105.38% for NVDA
466.89% for PLTR
-4.21% for TSLA
11.49% for ^GSPC
10.39% for ^IXIC




In chosen period 2024-01-01 to 2025-04-23 it was better to:
Make a one-time purchase of BTC-USD.
Don't buy ETH-USD and stay in dollar.
Make a one-time purchase of GC=F.
Make a one-time purchase of MSTR.
Make a one-time purchase of NVDA.
Make a one-time purchase of PLTR.
Buy TSLA with DCA strategy.
Make a one-time purchase of ^GSPC.
Make a one-time purchase of ^IXIC.
